In [30]:
#from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
"""
import PyPDF2

filepath1 = "GS.06.52850_Volume_swelling_of_FFKM_FKM_and_HNBR_elastomers_in_Saraline_Paraffins.pdf"
pdfFileObj = open(filepath1, 'rb') 
pdfreader = PyPDF2.PdfFileReader(pdfFileObj) 
count1 = pdfreader.numPages

text1 = []
for j in range(count1):
    page = pdfreader.getPage(j)
    pp = page.extractText()
    text1.append(pp)

#print(text1)

"""

In [4]:
#For text files

#f = open('C:/Users/Ranja.Sarkar/Chemical compatibility Elastomers and Metals.pdf')
#doc1 = str(f.read())

#f = open('C:/Users/Ranja.Sarkar/DEP 30.10.02.13 (Spec) Non-Metallic Materials - Selection and Application - 2003.pdf')
#doc2 = str(f.read())


In [31]:
import string
import nltk
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
     return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
     return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


In [32]:
## TF-IDF is for search query relevance

text1 = 'There is no change in hardness under ambient temperatures.'
text2 = 'There is change in hardness upon exposure to the solvent.'
text3 = 'There is change in hardness upon high pressure.'
test_set = [text1, text2, text3]

In [33]:
#LemVectorizer = CountVectorizer(tokenizer = LemNormalize, stop_words = 'english')
LemVectorizer = CountVectorizer(tokenizer = LemNormalize)
LemVectorizer.fit_transform(test_set)

print(LemVectorizer.vocabulary_), print(len(LemVectorizer.vocabulary_))

{'there': 12, 'is': 6, 'no': 7, 'change': 1, 'in': 5, 'hardness': 3, 'under': 14, 'ambient': 0, 'temperature': 10, 'upon': 15, 'exposure': 2, 'to': 13, 'the': 11, 'solvent': 9, 'high': 4, 'pressure': 8}
16


(None, None)

In [34]:
tf_matrix = LemVectorizer.transform(test_set).toarray()
print(tf_matrix)

tf_matrix.shape

[[1 1 0 1 0 1 1 1 0 0 1 0 1 0 1 0]
 [0 1 1 1 0 1 1 0 0 1 0 1 1 1 0 1]
 [0 1 0 1 1 1 1 0 1 0 0 0 1 0 0 1]]


(3, 16)

In [35]:
#tf-idf = tf x idf
#idf(t) = log((n+1)/(df(d,t)+1)) + 1 ##n = total number of documents, df = number of documents in which term t appears ##prevents zero divisions

#generally speaking, certain terms do occur more than others
#tf-idf scales up the importance of rarer terms and scales down the importance of more frequent terms 
#relative to the whole corpus

In [ ]:
#import math
#def idf(n,df):
#    result = math.log((n+1.0)/(df+1.0)) + 1
#    return result
#print("The idf for terms that appear in one document: " + str(idf(4,1)))
#print()"The idf for terms that appear in two documents: " + str(idf(4,2)))

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidfTran = TfidfTransformer(norm = "l2")
tfidf = tfidfTran.fit(tf_matrix)
#print(tfidf)
print(tfidfTran.idf_)


[1.69314718 1.         1.69314718 1.         1.69314718 1.
 1.         1.69314718 1.69314718 1.69314718 1.69314718 1.69314718
 1.         1.69314718 1.69314718 1.28768207]


In [25]:
#cosine similarity matrix is obtained by multiplying the if-idf matrix by its transpose
tfidf_matrix = tfidfTran.transform(tf_matrix)
print(tfidf_matrix.toarray())


[[0.41724161 0.24642961 0.         0.24642961 0.         0.24642961
  0.24642961 0.41724161 0.         0.         0.41724161 0.
  0.24642961 0.         0.41724161 0.        ]
 [0.         0.23488735 0.39769885 0.23488735 0.         0.23488735
  0.23488735 0.         0.         0.39769885 0.         0.39769885
  0.23488735 0.39769885 0.         0.30246022]
 [0.         0.28407693 0.         0.28407693 0.48098405 0.28407693
  0.28407693 0.         0.48098405 0.         0.         0.
  0.28407693 0.         0.         0.36580076]]


In [26]:
cos_similarity_matrix = (tfidf_matrix * tfidf_matrix.T).toarray()
print(cos_similarity_matrix)


[[1.         0.28941599 0.35002483]
 [0.28941599 1.         0.44427055]
 [0.35002483 0.44427055 1.        ]]


In [ ]:
#Similarity matrix says that text2 and text3 are more similar to each other than other pairs

# Combination of CountVectorizer & TfidfTransformer (TfidfVectorizer)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

#StopWords = stopwords.words('english')

In [29]:
#TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words = 'english')
TfidfVec = TfidfVectorizer(tokenizer = LemNormalize)

def cos_similarity(textlist):
    tfidf = TfidfVec.fit_transform(textlist)
    return (tfidf * tfidf.T).toarray()
cos_similarity(test_set)

array([[1.        , 0.28941599, 0.35002483],
       [0.28941599, 1.        , 0.44427055],
       [0.35002483, 0.44427055, 1.        ]])